## The validation set approach

In [1]:
require(ISLR)

Loading required package: ISLR


In [2]:
set.seed(1)

In [3]:
#work on Auto dataset
#make a sample of integers between 1,392 of size  196  the half of Auto dataset
train<-sample(392,196)
#linear regresssion trained on train set  (0.5 of whole dataset)
lm.fit<-lm(mpg~horsepower,data=Auto,subset=train)

#use predict() to estimate response on validation set (-train)
attach(Auto)
print(paste("estimated MSE on lm:",mean( (mpg-predict(lm.fit,Auto))[-train]^2  )))

[1] "estimated MSE on lm: 26.1414211520072"


In [4]:
#do the estimation of the same MSE on poly models
lm.fit2<-lm(mpg~poly(horsepower,2),data=Auto,subset=train)
print(paste("estimated MSE on lm poly 2:",mean( (mpg-predict(lm.fit2,Auto))[-train]^2  )))

lm.fit3<-lm(mpg~poly(horsepower,3),data=Auto,subset=train)
print(paste("estimated MSE on lm poly 3:",mean( (mpg-predict(lm.fit3,Auto))[-train]^2  )))

lm.fit4<-lm(mpg~poly(horsepower,4),data=Auto,subset=train)
print(paste("estimated MSE on lm poly 4:",mean( (mpg-predict(lm.fit4,Auto))[-train]^2  )))


[1] "estimated MSE on lm poly 2: 19.8225850408262"
[1] "estimated MSE on lm poly 3: 19.7825166856023"
[1] "estimated MSE on lm poly 4: 19.999685962002"


In [5]:
##lets see how choosing another trainig set afects  MSE results
set.seed(2)
train<-sample(392,196) #resample
lm.fit<-lm(mpg~horsepower,data=Auto,subset=train)
print(paste("estimated MSE on lm:",mean( (mpg-predict(lm.fit,Auto))[-train]^2  )))
lm.fit2<-lm(mpg~poly(horsepower,2),data=Auto,subset=train)
print(paste("estimated MSE on lm poly 2:",mean( (mpg-predict(lm.fit2,Auto))[-train]^2  )))
lm.fit3<-lm(mpg~poly(horsepower,3),data=Auto,subset=train)
print(paste("estimated MSE on lm poly 3:",mean( (mpg-predict(lm.fit3,Auto))[-train]^2  )))
lm.fit4<-lm(mpg~poly(horsepower,4),data=Auto,subset=train)
print(paste("estimated MSE on lm poly 4:",mean( (mpg-predict(lm.fit4,Auto))[-train]^2  )))

[1] "estimated MSE on lm: 23.2955851508862"
[1] "estimated MSE on lm poly 2: 18.9012408317778"
[1] "estimated MSE on lm poly 3: 19.2573982608642"
[1] "estimated MSE on lm poly 4: 20.3853777964338"


## Leave-One-Out Cross-Validation (LOOCV)

In [6]:
require(boot)
glm.fit<-glm(mpg~horsepower,data=Auto) #no params in glm  == linear regression is done
cv.err<-cv.glm(Auto,glm.fit) 
print("cv error on lm:")
print(cv.err$delta)

Loading required package: boot


[1] "cv error on lm:"
[1] 24.23151 24.23114


In [7]:
#redo loocv for 5 degrees of polynomial 
cv.error<-rep(0,5)
for (i in 1:5){
	print(paste("loocv for degree:",i,".."))
	glm.fit<-glm(mpg~poly(horsepower,i),data=Auto)
	cv.error[i] <- cv.glm(Auto, glm.fit)$delta[1]
}
print("loocv for poly of 5 degrees")
print(cv.error)

[1] "loocv for degree: 1 .."
[1] "loocv for degree: 2 .."
[1] "loocv for degree: 3 .."
[1] "loocv for degree: 4 .."
[1] "loocv for degree: 5 .."
[1] "loocv for poly of 5 degrees"
[1] 24.23151 19.24821 19.33498 19.42443 19.03321


## K-fold Cross Validation 

In [8]:
## redo the previous example with k-fold validation with k=10
#faster cpu execution
set.seed(17)
cv.error10<-rep(0,10)
for (i in 1:10){
	print(paste("k-fold for degree:",i,".."))
	glm.fit<-glm(mpg~poly(horsepower,i),data=Auto)
	cv.error10[i] <- cv.glm(Auto, glm.fit,K=10)$delta[1] #same cv.glm func is used with K param
}
print("k-fold for poly of 10 degrees k = 10")
print(cv.error10)

#we confirm that there is no point to use  the polynomial degree higher than 2 
#in the linear regression model

[1] "k-fold for degree: 1 .."
[1] "k-fold for degree: 2 .."
[1] "k-fold for degree: 3 .."
[1] "k-fold for degree: 4 .."
[1] "k-fold for degree: 5 .."
[1] "k-fold for degree: 6 .."
[1] "k-fold for degree: 7 .."
[1] "k-fold for degree: 8 .."
[1] "k-fold for degree: 9 .."
[1] "k-fold for degree: 10 .."
[1] "k-fold for poly of 10 degrees k = 10"
 [1] 24.20520 19.18924 19.30662 19.33799 18.87911 19.02103 18.89609 19.71201
 [9] 18.95140 19.50196


## Bootstrap

In [9]:
#define a function which returns alpha investment coefficient
#works with Portfolio dataset
alpha.fn<-function(data,index){
	x<-data$X[index]
	y<-data$Y[index]
    vx<-var(x)
    vy<-var(y)
    cxy<-cov(x,y)
    return( (vx-cxy)/(vx+vy-2*cxy) )
}

print(alpha.fn(Portfolio,1:100))


[1] 0.4241679


In [10]:
set.seed(1)
#alpha from a sampled dataset
print(alpha.fn(Portfolio,sample(100,100,replace=T)))

#calling  a bootstrap 1000 times
print(boot(Portfolio, alpha.fn, R=1000))


[1] 0.4036167

ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.4241679 7.315422e-05  0.08861826


### Estimate accuracy of the linear regression model

In [11]:
#define a function which returns coeffcients of our original lm model, using a specific 
#dataset index
boot.fn<-function(data,index){
	return (coef(lm(mpg~horsepower,data=data,subset=index)))
}
#return coeficients from an original dataset's index
print(boot.fn(Auto,1:392))

(Intercept)  horsepower 
 39.9358610  -0.1578447 


In [12]:
set.seed(1)
print("## applying bootstrap 1000 times to estimate lm coefficients and its variability")
boot(Auto,boot.fn,1000)

[1] "## applying bootstrap 1000 times to estimate lm coefficients and its variability"



ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0269563085 0.859851825
t2* -0.1578447 -0.0002906457 0.007402954

In [13]:
#compare this with a standard lm call
lm.fit<-lm(mpg~horsepower,data=Auto)
print(summary(lm.fit)$coef)

#we can observe the coefficients and their standard erors are similar

              Estimate  Std. Error   t value      Pr(>|t|)
(Intercept) 39.9358610 0.717498656  55.65984 1.220362e-187
horsepower  -0.1578447 0.006445501 -24.48914  7.031989e-81
